#  Download Pokemon sprites

## Functions for accessing PokéAPI

In [1]:
import requests
import json

def get_pokeapi(path):
    request = requests.get(f'https://pokeapi.co/api/v2/{path}')
    response = json.loads(request.text)
    return response

def get_pokemon(id_or_name):
    return get_pokeapi(f'pokemon/{id_or_name}')

## Functions for extracting information from API responses

In [2]:
import jsonpath_ng

def find_values_in(object, path):
    match = path.find(object)
    values = list(map(lambda o: o.value, match))
    return values

TYPE_PATH = jsonpath_ng.parse('$.types[*].type.name')

def get_types_of(pokemon):
    return find_values_in(pokemon, TYPE_PATH)

ID_PATH = jsonpath_ng.parse('$.id')

def get_id_of(pokemon):
    return find_values_in(pokemon, ID_PATH)[0]

## Functions for downloading and storing sprites

In [3]:
import requests
import os

def create_directories_if_necessary(file):
    if not os.path.exists(os.path.dirname(file)):
        os.makedirs(os.path.dirname(file))

def download_file_to(url, location):
    if os.path.exists(location):
        return
    request = requests.get(url)
    create_directories_if_necessary(location)
    with open(location, 'wb') as file:
        file.write(request.content)

def download_sprite_for(pokemon, sub_directory):
    id = get_id_of(pokemon)
    url = f'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/{id}.png'
    location = f'sprites/{sub_directory}/{id}.png'
    download_file_to(url, location)

## Functions for progress report
(*stolen from [here](https://www.mikulskibartosz.name/how-to-display-a-progress-bar-in-jupyter-notebook/)*)

In [4]:
import sys
from IPython.display import clear_output

def clamp(n, minimum, maximum):
    return max(minimum, min(n, maximum))

def update_progress(progress):
    bar_length = 20
    progress = clamp(progress, 0, 1)
    block = int(round(bar_length * progress))
    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

## Download sprites

In [5]:
NUMBER_OF_POKEMON = 807

GRASS_TYPE = 'grass'
FIRE_TYPE = 'fire'
WATER_TYPE = 'water'

id = 1
update_progress(0)
while id <= NUMBER_OF_POKEMON:
    pokemon = get_pokemon(id)
    types = get_types_of(pokemon)
    if GRASS_TYPE in types:
        download_sprite_for(pokemon, GRASS_TYPE)
    if FIRE_TYPE in types:
        download_sprite_for(pokemon, FIRE_TYPE)
    if WATER_TYPE in types:
        download_sprite_for(pokemon, WATER_TYPE)
    id += 1
    update_progress(id / NUMBER_OF_POKEMON)

print('done!')

Progress: [####################] 100.0%
done!
